## Machine Learning Model - Using Neural Network Model to Make a Prediction of Mag and Depth


In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

import os
print(os.listdir("../machine_learning-project"))

['.git', '.gitignore', '.ipynb_checkpoints', 'data_base', 'ML_Earthquake_Neuron.ipynb', 'models', 'README.md']


In [2]:
# Loading the Data

data = pd.read_csv("data_base/earthquakes.csv")
data.head()

,date,latitude,longitude,depth,mag,id,place
0,19600102,-55.877,-1.890,15.0,6.3,iscgem877909,Bouvet Island region
1,19600103,43.700,84.542,15.0,5.7,iscgem877920,"northern Xinjiang, China"
2,19600104,11.374,42.609,15.0,6.1,iscgemsup877930,Djibouti
3,19600104,45.069,26.829,40.0,5.4,iscgemsup877933,Romania
4,19600107,6.418,94.756,15.0,5.6,iscgem877954,"Nicobar Islands, India region"


In [3]:
#Displaying the Columns

data.columns

Index(['date', 'latitude', 'longitude', 'depth', 'mag', 'id', 'place'], dtype='object')

In [4]:
data = data[['date','latitude', 'longitude', 'depth', 'mag']]
data.head()

,date,latitude,longitude,depth,mag
0,19600102,-55.877,-1.890,15.0,6.3
1,19600103,43.700,84.542,15.0,5.7
2,19600104,11.374,42.609,15.0,6.1
3,19600104,45.069,26.829,40.0,5.4
4,19600107,6.418,94.756,15.0,5.6


In [5]:
# import datetime
# import time

# timestamp = []
# for d, t in zip(data['Date'], data['Time']):
#     try:
#         ts = datetime.datetime.strptime(d+' '+t, '%m/%d/%Y %H:%M:%S')
#         timestamp.append(time.mktime(ts.timetuple()))
#     except ValueError:
#         # print('ValueError')
#         timestamp.append('ValueError')

In [6]:
# timeStamp = pd.Series(timestamp)
# data['Timestamp'] = timeStamp.values

In [7]:
# #Dropping the columns not needed

# final_data = data.drop(['Date', 'Time'], axis=1)
# final_data = final_data[final_data.Timestamp != 'ValueError']
# final_data.head()

Model Splitting 

In [8]:
#Splitting the data

X = data[['date', 'latitude', 'longitude']]
y = data[['mag', 'depth']]

In [9]:
#Splitting Xs and ys into traing and test datasets 

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

(60679, 3) (15170, 3) (60679, 2) (15170, 3)


In [10]:
#from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train)
reg.predict(X_test)

array([[  5.326 , 566.1304],
       [  5.572 ,  39.682 ],
       [  5.205 ,  37.069 ],
       ...,
       [  5.43  ,  43.37  ],
       [  5.594 ,  11.25  ],
       [  5.149 ,  33.21  ]])

In [11]:
#

reg.score(X_test, y_test)


0.45738825747372863

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[10, 20, 50, 100, 200, 500]}

grid_obj = GridSearchCV(reg, parameters)
grid_fit = grid_obj.fit(X_train, y_train)
best_fit = grid_fit.best_estimator_
best_fit.predict(X_test)

array([[  5.329  , 565.74184],
       [  5.4768 ,  39.8714 ],
       [  5.3046 ,  36.45974],
       ...,
       [  5.4344 ,  44.2416 ],
       [  5.6732 ,  11.4206 ],
       [  5.162  ,  33.486  ]])

In [13]:
#Creating the Best Fit 

best_fit.score(X_test, y_test)


0.4615339695577431

##Neural Network Model

In [14]:
best_fit.score(X_test, y_test)
from keras.models import Sequential
from keras.layers import Dense

def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [15]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

# neurons = [16, 64, 128, 256]
neurons = [16]
# batch_size = [10, 20, 50, 100]
batch_size = [10]
epochs = [10]
# activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'exponential']
activation = ['sigmoid', 'relu']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizer = ['SGD', 'Adadelta']
loss = ['squared_hinge']

param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs, activation=activation, optimizer=optimizer, loss=loss)

In [16]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.800000 using {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}
0.000000 (0.000000) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.800000 (0.400000) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}
0.000000 (0.000000) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.800000 (0.400000) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}


In [17]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(3,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1, validation_data=(X_test, y_test))

Epoch 1/20
6068/6068 [==============================] - 6s 951us/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 2/20
6068/6068 [==============================] - 6s 956us/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 3/20
6068/6068 [==============================] - 13s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 4/20
6068/6068 [==============================] - 14s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 5/20
6068/6068 [==============================] - 13s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 6/20
6068/6068 [==============================] - 13s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 7/20
6068/6068 [==============================] - 13s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_

In [19]:
[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

475/475 [==============================] - 1s 2ms/step - loss: 0.5028 - accuracy: 0.9848
Evaluation result on Test Data : Loss = 0.5028371810913086, accuracy = 0.9848384857177734


In [20]:
model.save('models/ml_model_fulldata.h5')

# Loading a Model

In [21]:
# Load the model
from tensorflow.keras.models import load_model
eq_model = load_model("models/ml_model_fulldata.h5")

## Evaluating the loaded model


In [ ]:
model_accuracy = eq_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network -  Accuracy: {model_accuracy}")